In [24]:
!jupyter nbconvert --to=python
!pip install PyDrive
!pip install gensim matplotlib nltk numpy pandas pyLDAvis scikit_learn spacy seaborn gensim  twint PyDrive
!pip install gensim
!pip install pyldavis
!python -m spacy download en
!pip install twint
!pip install nltk

This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    Execute the notebook prior to export.
    Equivalent to: [--ExecutePr

In [25]:
import gensim
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
from datetime import timedelta
import datetime
import nest_asyncio
import pandas as pd
import spacy
import re
import nltk
import twint
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nest_asyncio.apply()
nltk.download('wordnet')
wnl = WordNetLemmatizer()
nlp = spacy.load('en_core_web_sm')
nltk.download('stopwords')


[nltk_data] Downloading package wordnet to /Users/m/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/m/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:

# Set the limit of tweets to be scraped
limit_of_tweets = 100
def run_twint():

    #since_in_day = 5
    #since = (datetime.datetime.now() - timedelta(since_in_day)).strftime('%Y-%m-%d')
    c = twint.Config()
    c.Search = 'Police'
    c.Limit = limit_of_tweets
    c.Store_csv = True
    #c.Since = since
    c.Output = 'output.csv'
    c.Pandas = True
    c.Pandas_clean = True
    c.Lang = "en"
    twint.run.Search(c)
    df = twint.storage.panda.Tweets_df 
    return df


stop_words = set(stopwords.words('english'))
#data = pd.read_csv('/Users/m/Documents/GitHub/Data_Mining/data/Complete.csv')
data = run_twint()


1464475938318479361 2021-11-27 01:07:26 -0500 <aaugh> The Police - Every Breath You Take (Master Chic Mix)  https://t.co/uzPRYVlrvx via @YouTube . @BarackObama @katyperry @DalaiLama @Pontifex @justinbieber @TheEllenShow @AOC @UN @taylorswift13 @POTUS @VP @hkfp @CGTNOfficial @globaltimesnews
1464475931783962626 2021-11-27 01:07:25 -0500 <Proiqra> Police were deployed outside Nasla Tower  https://t.co/wjucZt7fcy
1464475924695375877 2021-11-27 01:07:23 -0500 <DelhiPolice> @ABHISHEKishere @LtGovDelhi @HMOIndia @CPDelhi @cp_delhi @DCPSEastDelhi @Saurabh_MLAgk @PunjabKesariCom The said video is almost a year old. Based on that, legal action was taken against the culprits in year 2020 itself with four arrests. Portraying it as a current event is misrepresentation of facts. #DelhiPolice has zero tolerance towards any form of organized crime.
1464475917959503875 2021-11-27 01:07:22 -0500 <Busines65959783> Sacked – Disgraced West Midlands police officers who have lost their jobs  https://t.co/Xn

In [27]:
import string
# Cleaning the tweets Step 2
def clean_tweets_tb(input):
    punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'
    text = str(input)
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub("@[A-Za-z0-9]+", "", text)
    text = re.sub(r"@[A-Za-z0-9]+", "", text)
    text = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text)
    text = re.sub(r"_[A-Za-z0-9]+", "", text)
    text = re.sub(r"__", "", text)
    text = re.sub(' +', ' ', text)
    text = re.sub('[' + punctuation + ']+', ' ', text)  # strip punctuation
    text = re.sub('\s+', ' ', text)  # remove double spacing
    text = re.sub('’', ' ', text)  # remove apostrophe'
    text = re.sub('\'', ' ', text)  # remove double spacing
    text = re.sub('([0-9]+)', '', text)  # remove numbers
    text = "".join([char for char in text if char not in string.punctuation])
    text = text.lower()  # Lower text
    return text

<>:14: DeprecationWarning: invalid escape sequence \s
<>:14: DeprecationWarning: invalid escape sequence \s
/var/folders/7_/6p6y3gn11wx0939j26kng5rh0000gn/T/ipykernel_82851/1348241827.py:14: DeprecationWarning: invalid escape sequence \s
  text = re.sub('\s+', ' ', text)  # remove double spacing


In [28]:
import pandas as pd
pd.set_option("display.max_colwidth", 200)
import numpy as np
import json
import re
import gzip
import spacy

import gensim
from gensim import corpora

import pyLDAvis
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [29]:
def remove_content(text):
    text = re.sub(r"http\S+", "", text) #remove urls
    text=re.sub(r'\S+\.com\S+','',text) #remove urls
    text=re.sub(r'\@\w+','',text) #remove mentions
    text =re.sub(r'\#\w+','',text) #remove hashtags
    return text
def process_text(text, stem=False): #clean text
    text=remove_content(text)
    text = re.sub('[^A-Za-z]', ' ', text.lower()) #remove non-alphabets
    tokenized_text = word_tokenize(text) #tokenize
    clean_text = [
         word for word in tokenized_text
         if word not in stop_words
    ]
    if stem:
        clean_text=[stemmer.stem(word) for word in clean_text]
    return ' '.join(clean_text)

In [32]:
data['cleaned_tweets']= [process_text(tweet) for tweet in data['tweet']]
data['cleaned_tweets']= [remove_content(tweet) for tweet in data['tweet']]

data['cleaned_tweets'].values
print(data['cleaned_tweets'])

0                                                                                                                                   The Police - Every Breath You Take (Master Chic Mix)   via  .              
1                                                                                                                                                                    Police were deployed outside Nasla Tower  
2               The said video is almost a year old. Based on that, legal action was taken against the culprits in year 2020 itself with four arrests. Portraying it as a current event is misrepresentation...
3                                                                                                                                   Sacked – Disgraced West Midlands police officers who have lost their jobs  
4        Hello ‼️  we need extra armed police and security around the stadium and inside stadium for the concerts and please check right now for this person ‼️ this cou

In [33]:
#PREPARE
r = [process_text(x,stem=False).split() for x in data['tweet'].tolist()] 
dictionary = corpora.Dictionary(r)
corpus = [dictionary.doc2bow(rev) for rev in r]#initialize model and print topics
doc_term_matrix = [dictionary.doc2bow(rev) for rev in r]
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=7, random_state=100,
                chunksize=1000, passes=50)
lda_model.print_topics()

vis = gensimvis.prepare(lda_model, doc_term_matrix, dictionary)

pyLDAvis.enable_notebook()
vis

/usr/local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/usr/local/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is dep

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.133449 -0.072760       1        1  24.548951
4     -0.131488 -0.071732       2        1  23.570749
0     -0.125335 -0.058946       3        1  18.111135
3     -0.119229 -0.055442       4        1  16.329375
1     -0.035133  0.344416       5        1   7.790527
2      0.307185 -0.010458       6        1   5.845988
5      0.237448 -0.075077       7        1   3.803274, topic_info=        Term        Freq       Total Category  logprob  loglift
1169   pcmax  111.000000  111.000000  Default  30.0000  30.0000
430       de   68.000000   68.000000  Default  29.0000  29.0000
1168       n   52.000000   52.000000  Default  28.0000  28.0000
1166      dn   50.000000   50.000000  Default  27.0000  27.0000
436        j   50.000000   50.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
1600       l   11.896416   39.518522   Topic7  -3.7066   2.0688
1070      gi    0.276565    1.120957   Topic7  -7.4681   1.8698
292   street    0.276284    1.261994   Topic7  -7.4691   1.7503
439       la   11.645399   53.249618   Topic7  -3.7279   1.7492
5     police   13.457224  517.738986   Topic7  -3.5833  -0.3806

[352 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
4568      7  0.971986     ables
944       3  0.178777  accident
944       4  0.715109  accident
535       2  0.640175       act
535       3  0.213392       act
...     ...       ...       ...
202       3  0.694323        ya
500       1  0.725587     years
500       2  0.207311     years
1017      1  0.812839       yet
370       6  0.933453      zone

[452 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 1, 4, 2, 3, 6])